## 26. Remove Duplicates from Sorted Array

In [ ]:
(defun removeduplicates (nums)
  (length (remove-duplicates nums)))

## 27. Remove Element

In [ ]:
;;; 使用 remove 或 delete 都可以做到，但因为不能使用额外的空间，
;;; 那么可以有副作用的 delete，但原数组最终结果是未知的，不可行。
;;;
;;; 使用 car 和 pop，pop 是原地修改，但在函数中却不行。
;;;
;;; setf 或 rotatef 宏可以做到，最终使用双指针。
(defun remove-element (nums val)
  (let ((n (length nums))
        (i 0))
    (loop while (< i n) do
         (if (= (nth i nums) val)
             (rotatef (nth i nums)
                      (nth (decf n) nums))
             (incf i)))
    n))

## 28. Implement strStr()

In [ ]:
(defun strstr (haystack needle)
  (let ((needle-size (length needle)))
    (loop for i to (- (length haystack)
                      needle-size) do
         (if (string= (subseq haystack i (+ i needle-size))
                      needle)
             (return-from strstr i)))
    -1))

## 29. Divide Two Integers

In [ ]:
;;; 不能用乘、除、求模运算符。
;;; 只用加减的话，时间复杂度与商的大小成正比。
;;; 但位移可以实现类似乘除的操作。
(defun divide (dividend divisor)
  (let ((positive (and (> dividend 0)
                       (> divisor 0)))
        (dividend (abs dividend))
        (divisor (abs divisor))
        (res 0))
    (loop while (> dividend divisor) do
         (let ((temp divisor)
               (m 1))
           (loop while (> dividend temp) do
                (decf dividend temp)
                (incf res m)
                (setf temp (ash temp 1))
                (setf m (ash m 1)))))
    (if (not positive)
        (setf res (- res)))
    (min (max res (- (expt 2 31))) (1- (expt 2 31)))))

## 30. Substring with Concatenation of All Words

In [ ]:
(defun find-substring (s words)
  (let ((word-length (length (car words)))
        (words-size (length words))
        (hash-table (make-hash-table :test 'equal))
        (res))
    ;; 使用 hash-table 计算 words 中每个字符串出现的次数
    (loop for word in words do
         (incf (gethash word hash-table 0)))
    (loop for i to (- (length s) (* word-length words-size)) do
         (let ((plist)) ; 使用属性表存储子串出现次数
           (loop for j below words-size do
                (let* ((start (+ i (* j word-length)))
                       (end (+ start word-length))
                       (substr (subseq s start end))
                       (key (intern substr "KEYWORD")))
                  (if (< (getf plist key 0)
                         (gethash substr hash-table -1))
                      (incf (getf plist key 0))
                      (return)))
              finally (push i res))))
    res))

## 31. Next Permutation

算法参考维基百科 - [全排列生成算法 - 字典序法](https://zh.wikipedia.org/wiki/%E5%85%A8%E6%8E%92%E5%88%97%E7%94%9F%E6%88%90%E7%AE%97%E6%B3%95#%E5%AD%97%E5%85%B8%E5%BA%8F%E6%B3%95)

设P是集合{1，2，……n-1，n}的一个全排列：P=P1P2……Pj-1PjPj+1……Pn（1≤P1，P2，……，Pn≤n-1）
1. 从排列的右端开始，找出第一个比右边数字小的数字的序号j，即j=max{i|Pi<Pi+1，i>j}
2. 在Pj的右边的数字中，找出所有比Pj大的数字中最小的数字Pk，即k=min{i|Pi>Pj，i>j}
3. 交换Pi，Pk
4. 再将排列右端的递减部分Pj+1Pj+2……Pn倒转，因为j右端的数字是降序，所以只需要其左边和右边的交换，直到中间，因此可以得到一个新的排列P'=P1P2……Pj-1PkPn……Pj+2Pj+1。

另可参考 [Next lexicographical permutation algorithm](https://www.nayuki.io/page/next-lexicographical-permutation-algorithm)

In [ ]:
;;; 下一个全排列，in-place
(defun next-permutation (nums)
  (let* ((size (length nums))
         ;; 从右端开始找出算法中的 j
         (j (loop for i from (- size 2) downto 0 do
                 (if (< (nth i nums)
                        (nth (1+ i) nums))
                     (return i)))))
    (if j
        ;; 找到 j 之后寻找算法中的 k
        (let* ((k (1+ j)))
          (loop for i from (1+ j) below size do
               (if (and (< (nth j nums)
                           (nth i nums))
                        (< (nth i nums)
                           (nth k nums)))
                   (setf k i)))
          (when (< (nth j nums)
                   (nth k nums))
            (rotatef (nth j nums) (nth k nums))
            (let ((start (1+ j))
                  (end (1- size)))
              (loop while (< start end) do
                   (rotatef (nth start nums) (nth end nums))
                   (incf start)
                   (decf end)))))
        ;; 未找到符合条件的 j，将输入逆序
        (let ((start 0)
              (end (1- size)))
          (loop while (< start end) do
               (rotatef (nth start nums) (nth end nums))
               (incf start)
               (decf end))))))

## 32. Longest Valid Parentheses

In [ ]:
;;; 刚开始想到用动态规划，后来发现直接使用栈就可以了，更简单清晰。
;;; Lisp 中的列表来代替实现栈，更加方便。
(defun longest-valid-parentheses (s)
  (let ((stack)
        (longest 0))
    (loop for c across s 
       for i to (length s) do
         (if (and (equal (car (car stack)) #\()
                  (equal c #\)))
             (progn
               (pop stack)
               (if (null stack)
                   (setf longest (1+ i))
                   (setf longest (- i (cdr (car stack))))))
             (push (cons c i) stack)))
    longest))

## 33. Search in Rotated Sorted Array

算法参考 [Clever idea making it simple](https://leetcode.com/problems/search-in-rotated-sorted-array/discuss/14435/Clever-idea-making-it-simple) 与 [Pretty short C++/Java/Ruby/Python](https://leetcode.com/problems/search-in-rotated-sorted-array/discuss/14419/Pretty-short-C%2B%2BJavaRubyPython)

假设查找的范围为 `lo` ~ `hi`，`mid` 为范围中间的索引。使 `target` 在 `mid` 右边（`lo = mid + `）的所有可能为：

1. nums[0] > target and (nums[0] > nums[mid] == target > nums[mid])
2. nums[0] < target and nums[0] > nums[mid] and target > nums[mid]

In [ ]:
;;; 
(defun search-rotated-sorted (nums target)
  (let ((lo 0)
        (hi (length nums)))
    (loop while (< lo hi) do
         (let ((mid (floor (+ lo hi) 2)))
           ;; 实现类似异或的操作，`logxor` 使用数值为参数，不符合要求。
           ;; 可使用 `reduce` 宏自己写一个作用于 `t` 与 `nil` 之上的异或操作
           ;; (reduce #'(lambda (x y) (not (equal x y))) sequence)
           (if (not (equal (not (equal (> (first nums) target)
                                       (> (first nums) (nth mid nums))))
                           (> target (nth mid nums))))
               (setf lo (1+ mid))
               (if (= (nth mid nums) target)
                   (return mid)
                   (setf hi mid))))
       finally (return -1))))

## 34. Find First and Last Position of Element in Sorted Array

先二分查找，定位到目标值 target，未找到直接返回 `[-1, -1]`，找到则从该位置两端找出第一个和最后一个目标值。

In [ ]:
(defun search-range (nums target)
  (let ((start 0)
        (end (length nums)))
    (loop while (< start end) do
         (let ((mid (floor (+ start end) 2)))
           (cond
             ((< (nth mid nums) target)
              (setf start (1+ mid)))
             ((> (nth mid nums) target)
              (setf end mid))
             (t
              ;; nums[mid] == target
              (let ((first-position mid)
                    (last-position mid))
                (loop while (and (> 0 first-position)
                                 (= target (nth (1- first-position) nums))) do
                     (decf first-position))
                (loop while (and (< last-position (length nums))
                                 (= target (nth (1+ last-position) nums))) do
                     (incf last-position))
                (return (list first-position last-position))))))
       finally (return '(-1 -1)))))

## 35. Search Insert Position

原数组中没有重复元素，二分查找要插入的位置，直至剩余最后一个数，若大于或等于 target，则返回该值所在位置，否则返回该值的前一个位置。

In [ ]:
(defun search-insert (nums target)
  (let ((lo 0)
        (hi (length nums)))
    (loop while (< lo hi) do
         (let ((mid (floor (+ lo hi) 2)))
           (cond 
             ((> (nth mid nums) target)
              (setf hi mid))
             ((< (nth mid nums) target)
              (setf lo (1+ mid)))
             (t
              (return mid))))
       finally (if (> (nth lo nums) target)
                   (return lo)
                   (return (1- lo))))))

## 36. Valid Sudoku

实际只要验证九横九列再加九个 `3x3` 方格即可。

In [ ]:
(defun not-valid-sudoku (box) ; 验证除 "." 外是否有重复
  (let ((digits (remove "." box :test #'string=)))
    (not (= (length digits)
            (length (remove-duplicates digits :test #'string=))))))

(defun is-valid-sudoku (board)
  (loop for i to 8 
     do (cond ((not-valid-sudoku (nth i board)) ; 验证行
               (return nil))
              ;; 验证列
              ((not-valid-sudoku (loop for row in board
                                    collect (nth i row)))
               (return nil))
              ;; 验证 3x3 方格，(x, y) 为 3x3 方格左上角顶点
              ((not-valid-sudoku (let ((x (*(floor i 3) 3))
                                       (y (* (mod i 3) 3)))
                                   (loop for row from x below (+ x 3)
                                      append (subseq (nth row board)
                                                     y
                                                     (+ y 3)))))
               (return nil)))
     finally (return t)))

## 37. Sudoku Solver

想法：

1. 先找出所有空白位置的所有可能值；
2. 遍历空白位置，当前空白位置的可能值与行、列、3x3 方格中其他空白位置的可能值作补集，只剩一个元素值的补集，即为该位置的值。

网上看了一下，回溯算法还没试过。

In [ ]:
(defun solve-sudoku (board)
  (let ((empty 0))
    (loop for row to 8
       do (loop for col to 8
             do (when (string= "."
                               (nth col (nth row board)))
                  (incf empty) ; 空白位置 +1
                  ;; 设置空白位置为所有可能取值组成的列表
                  (setf (nth col (nth row board))
                        (sudoku-probability board row col)))))
    (loop while (> empty 0)
       do (loop for row to 8
             do (loop for col to 8
                   do (when (equal (type-of (nth col (nth row board)))
                                   'cons)
                        ;; 该位置类型是列表时，更新其所有可能的取值
                        (setf (nth col (nth row board))
                              (sudoku-probability board row col))
                        ;; 列表长度为 1，该位置确定
                        (if (= (length (nth col (nth row board))) 1)
                            (progn
                              (setf (nth col (nth row board))
                                    (first (nth col (nth row board))))
                              (decf empty))
                            ;; 某一补集长度恰好为 1 时，该位置值确定
                            (let ((complement (single-sudoku-complement
                                               board
                                               row
                                               col)))
                              (when complement
                                (setf (nth col (nth row board))
                                      (first complement))
                                (decf empty))))))))))


(defun sudoku-boxes (board row col)
  "获取根据所在行列位置获取所在行、列、3x3 方格"
  (list
   ;; 行
   (nth row board)
   ;; 列
   (loop for board-row in board
      collect (nth col board-row))
   ;; 3x3 方格，(x, y) 为 3x3 方格左上角顶点
   (let ((x (* (floor row 3) 3))
         (y (* (floor col 3) 3)))
     (loop for row from x below (+ x 3)
        append (subseq (nth row board)
                       y
                       (+ y 3))))))


;;; 获取根据行和列获取该位置的所有可能取值，数独中可能有 "."、列表、数字字符串。
(defun sudoku-probability (board row col)
  "获取根据行和列获取该位置的所有可能取值"
  (set-difference
   '("1" "2" "3" "4" "5" "6" "7" "8" "9")
   (loop for cell in (apply #'append 
                            (sudoku-boxes board row col))
      unless (or (equal "." cell)
                 (equal (type-of cell)
                        'cons))
      collect cell)
   :test #'string=))


(defun single-sudoku-complement (board row col)
  "获取补集长度为 1 的列表中的元素，没有则返回 nil"
  (let ((current (nth col (nth row board))))
    (loop for box in (sudoku-boxes board row col)
       do (if (= (length (set-difference current box)) 1)
              (return (first (set-difference current box)))))))

## 38. Count and Say

一开始没明白题意，明白题目什么意思就知道怎么做了。

In [ ]:
(let ((say-table (make-hash-table)))
  ;; 使用散列表进行缓存
  (setf (gethash 1 say-table) "1")
  (defun count-and-say (n)
    (cond
      ((gethash n say-table)
       (gethash n say-table))
      (t
       (setf (gethash n say-table)
             (count-nums (count-and-say (1- n))))
       (gethash n say-table)))))


(defun count-nums (nums-string)
  "计算字符串中数字首先的次数，相邻并相同的为一组计算"
  (with-output-to-string (out)
    (let ((pre-num (elt nums-string 0))
          (count 0))
      (loop for num across nums-string
         do (if (string= num pre-num)
                (incf count)
                (progn
                  (format out "~a~a" count pre-num)
                  (setf count 1 pre-num num)))
         finally (format out "~a~a" count pre-num))
      out)))

## 39. Combination Sum

使用递归。

In [ ]:
(defun combination-sum (candidates target)
  (cond
    ((= target 0)
     (values nil t))
    ((< target 0)
     (values nil nil))
    (t
     (let ((res))
       (loop for c in candidates
          for start to (length candidates)
          do (multiple-value-bind (solutions has-solution)
                 (combination-sum (subseq candidates start) (- target c))
               ;; 有解
               (when has-solution
                 (if solutions
                     (loop for s in solutions do
                          (push (push c s) res))
                     ;; c 恰好为解
                     (push `(,c) res)))))
       (if res
           (values res t)
           (values nil nil))))))

## 40. Combination Sum II

跟上一题差不多，输出集有重复元素，但不能重复使用同一个元素，结果也不能有重复。

In [ ]:
(defun combination-sum-2 (candidates target)
  (cond
    ((= target 0)
     (values nil t))
    ((< target 0)
     (values nil nil))
    (t
     (let ((res))
       (loop for c in candidates
          for start from 1 to (length candidates)
          do (multiple-value-bind (solutions has-solution)
                 (combination-sum (subseq candidates start) (- target c))
               ;; 有解
               (when has-solution
                 (if solutions
                     (loop for s in solutions do
                          (push (push c s) res))
                     ;; c 恰好为解
                     (push `(,c) res)))))
       (if res
           (values (remove-duplicates
                    res
                    :test #'(lambda (x y) (not (set-difference x y))))
                   t)
           (values nil nil))))))